In [108]:
import numpy as np
import scipy.io as sio


input_3D = np.ones(shape=(128,128,128))

print(input_3D.shape)
print(type(input_3D))

(128, 128, 128)
<class 'numpy.ndarray'>


In [109]:
#takes images sized (128, 128, 128)
#outputs images sized (100, 81, 80)
#outputs from the center of the image
def roughChop(input_3D):
    if (input_3D.shape != (128, 128, 128)):
        print("You have passed an incorrectly sized array to roughChop. Required array size: (128, 128, 128).")
        print("You passed an array of size: " + str(input_3D.shape))
    else:
        output_3D = np.copy(input_3D)
        output_3D = output_3D[14:-14,24:-23,24:-24]
        return output_3D

In [110]:
input_3D_2 = np.zeros(shape=(100,100,1))
roughChop(input_3D_2)

You have passed an incorrectly sized array to roughChop. Required array size: (128, 128, 128).
You passed an array of size: (100, 100, 1)


In [111]:
output_3D = roughChop(input_3D)

In [112]:
print(output_3D.shape)

(100, 81, 80)



### The voxel intensities of each MRI and PET image are used for classification. The whole brain images are simply divided into 3 × 3 × 3 parts to extract 27 patches of size 50 × 41 × 40 voxels. Each patch has half overlaps with its neighbor in every direction.

In [113]:
#object which will yield 27 patches 

class patch_iter:
    #input requires a 3D image of size 100, 81, 80
    def __init__(self, input_3D):
        if (input_3D.shape != (100, 81, 80)):
            print("You have passed an incorrectly sized array to patch_iter. Required array size: (100, 81, 80).")
            print("You passed an array of size: " + str(input_3D.shape))
            return 
        self.i = 0
        self.n = 27
        self.input_3D = input_3D
    def __iter__(self):
        # Iterators are iterables too.
        # Adding this functions to make them so.
        return self
    def next(self):
        if self.i < self.n:
            i = self.i
            self.i += 1
            #print(i)
            #return i
            dim1_start_idxs = [0, 25, 50] #[0-49], [25-74], [50-99] inclusive
            dim1_end_idxs =[50, 75, 100]
            dim2_start_idxs = [0, 20, 40] #[0-39], [20-59], [41-80] inclusive, (slightly different overlap because 81 isn't even)
            dim2_end_idxs =[41, 61, 81]
            dim3_start_idxs = [0, 20, 40]
            dim3_end_idxs = [40, 60, 80]
            ii = int(i / 9)
            jj = int( (i %9)  / 3)
            kk = int(i % 3)
            print("dim 1: " + str(dim1_start_idxs[ii]) + " " + str(dim1_end_idxs[ii]), end ="\t")
            print("dim 2: " + str(dim2_start_idxs[jj]) + " " + str(dim2_end_idxs[jj]), end ="\t")
            print("dim 3: " + str(dim3_start_idxs[kk]) + " " + str(dim3_end_idxs[kk]), end ="\t")
            
            return input_3D[dim1_start_idxs[ii]:dim1_end_idxs[ii],dim2_start_idxs[jj]:dim2_end_idxs[jj],dim3_start_idxs[kk]:dim3_end_idxs[kk]]
        else:
            raise StopIteration()


In [114]:
y = patch_iter(output_3D)

for ii in range(27):
    thisPatch = y.next()
    print(thisPatch.shape)

dim 1: 0 50	dim 2: 0 41	dim 3: 0 40	(50, 41, 40)
dim 1: 0 50	dim 2: 0 41	dim 3: 20 60	(50, 41, 40)
dim 1: 0 50	dim 2: 0 41	dim 3: 40 80	(50, 41, 40)
dim 1: 0 50	dim 2: 20 61	dim 3: 0 40	(50, 41, 40)
dim 1: 0 50	dim 2: 20 61	dim 3: 20 60	(50, 41, 40)
dim 1: 0 50	dim 2: 20 61	dim 3: 40 80	(50, 41, 40)
dim 1: 0 50	dim 2: 40 81	dim 3: 0 40	(50, 41, 40)
dim 1: 0 50	dim 2: 40 81	dim 3: 20 60	(50, 41, 40)
dim 1: 0 50	dim 2: 40 81	dim 3: 40 80	(50, 41, 40)
dim 1: 25 75	dim 2: 0 41	dim 3: 0 40	(50, 41, 40)
dim 1: 25 75	dim 2: 0 41	dim 3: 20 60	(50, 41, 40)
dim 1: 25 75	dim 2: 0 41	dim 3: 40 80	(50, 41, 40)
dim 1: 25 75	dim 2: 20 61	dim 3: 0 40	(50, 41, 40)
dim 1: 25 75	dim 2: 20 61	dim 3: 20 60	(50, 41, 40)
dim 1: 25 75	dim 2: 20 61	dim 3: 40 80	(50, 41, 40)
dim 1: 25 75	dim 2: 40 81	dim 3: 0 40	(50, 41, 40)
dim 1: 25 75	dim 2: 40 81	dim 3: 20 60	(50, 41, 40)
dim 1: 25 75	dim 2: 40 81	dim 3: 40 80	(50, 41, 40)
dim 1: 50 100	dim 2: 0 41	dim 3: 0 40	(50, 41, 40)
dim 1: 50 100	dim 2: 0 41	dim 3: 2

In [115]:
import numpy as np

import scipy.io as sio

In [118]:
#def load_data(patch_i,data_i):
def load_data():
    #patch_i is an index for which patch in the current brain sample, so it's 0 to 26
    #data_i is an index for which brain sample, so it's 0 to num_brains, in the for loop, it is passed in as the index of the current iteration of the loop
    #data_i = data_i+1 #incrementing data_i by 1 each time, so i think that it goes from 1 to num_brains+1 entries when pulling patch data
    #dataSetpath = '/media/disk_e/PET_PatchDATA' #specified path to the location of the PET scan patch data, WE WILL NEED TO CHANGE THIS!
    #filename = 'AffineADNC64_Patch'+str(patch_i)+'_PET_'+str(data_i)+'.mat' #file has to be a .mat file, and, for them
                                                                            #it's in the format AffineADNC64_Patch(patch id)_PET_(data id).mat
    #full_filename= os.path.join(dataSetpath,filename) #creates the full conjoined path of the dataSet's path and the file's name
    full_filename = "matlab_output.mat"
    dataMat = sio.loadmat(full_filename) #loading the data(.mat) file
    AD_name = 'PatchAD' #column name for the alzheimer's detection data
    AD_data = dataMat[AD_name] #getting the alzheimer's detection patch from the data file
    num_AD =  AD_data.shape[0]
    NC_name = 'PatchNC' #column name for the normal control data
    NC_data = dataMat[NC_name]
    num_NC =  NC_data.shape[0]
    X_size,Y_size,Z_size = NC_data.shape[1:]
    del dataMat
    numAllSample = num_AD+num_NC
    adLabel =  np.ones((num_AD,))
    normalLabel =  np.zeros((num_NC,))
    imgADNC_3D = np.zeros((numAllSample,50,41,40))
    imgADNC_3D[:num_AD,:X_size,:Y_size,:Z_size] = AD_data[:,:50,:41,:40]
    imgADNC_3D[num_AD:,:X_size,:Y_size,:Z_size] = NC_data[:,:50,:41,:40]
    imgVector = imgADNC_3D.reshape(numAllSample,-1)
    imgVector = imgVector.astype('float16')
    imgVector = (imgVector-np.min(imgVector))*2.0/(float(np.max(imgVector)-np.min(imgVector)))-1
    data3dvt = imgVector
    imgLabel = np.hstack((adLabel,normalLabel))
    return data3dvt,imgLabel 

In [119]:
def save_data(array_of_3D_AD_patches, array_of_3D_NC_patches):
    file_name = "matlab_output"
    mdict = {'PatchAD': array_of_3D_AD_patches, 'PatchNC': array_of_3D_NC_patches}
    sio.savemat(file_name, mdict)

In [120]:
y = patch_iter(output_3D)

arrayOfPatches = np.zeros(shape=(27,50,41,40))

for ii in range(27):
    thisPatch = y.next()
    arrayOfPatches[ii] = thisPatch
    #print(thisPatch.shape)
    
print(arrayOfPatches.shape)
print(arrayOfPatches[1])




dim 1: 0 50	dim 2: 0 41	dim 3: 0 40	dim 1: 0 50	dim 2: 0 41	dim 3: 20 60	dim 1: 0 50	dim 2: 0 41	dim 3: 40 80	dim 1: 0 50	dim 2: 20 61	dim 3: 0 40	dim 1: 0 50	dim 2: 20 61	dim 3: 20 60	dim 1: 0 50	dim 2: 20 61	dim 3: 40 80	dim 1: 0 50	dim 2: 40 81	dim 3: 0 40	dim 1: 0 50	dim 2: 40 81	dim 3: 20 60	dim 1: 0 50	dim 2: 40 81	dim 3: 40 80	dim 1: 25 75	dim 2: 0 41	dim 3: 0 40	dim 1: 25 75	dim 2: 0 41	dim 3: 20 60	dim 1: 25 75	dim 2: 0 41	dim 3: 40 80	dim 1: 25 75	dim 2: 20 61	dim 3: 0 40	dim 1: 25 75	dim 2: 20 61	dim 3: 20 60	dim 1: 25 75	dim 2: 20 61	dim 3: 40 80	dim 1: 25 75	dim 2: 40 81	dim 3: 0 40	dim 1: 25 75	dim 2: 40 81	dim 3: 20 60	dim 1: 25 75	dim 2: 40 81	dim 3: 40 80	dim 1: 50 100	dim 2: 0 41	dim 3: 0 40	dim 1: 50 100	dim 2: 0 41	dim 3: 20 60	dim 1: 50 100	dim 2: 0 41	dim 3: 40 80	dim 1: 50 100	dim 2: 20 61	dim 3: 0 40	dim 1: 50 100	dim 2: 20 61	dim 3: 20 60	dim 1: 50 100	dim 2: 20 61	dim 3: 40 80	dim 1: 50 100	dim 2: 40 81	dim 3: 0 40	dim 1: 50 100	dim 2: 40 81	dim 3: 20 60	dim 1

In [121]:
input_3D_AD = arrayOfPatches
input_3D_NC = arrayOfPatches[:5] + 1

save_data(input_3D_AD, input_3D_NC)

In [103]:
data3dvt,imgLabel = load_data()

In [122]:
print(data3dvt.shape)
print(imgLabel)

length_v = imgLabel.shape[0]        
data3dvt_reshaped = data3dvt.reshape(length_v, 50, 41, 40)
print(data3dvt_reshaped.shape)

(32, 82000)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 0. 0. 0.]
(32, 50, 41, 40)


In [123]:
# take all dcm files in a folder and put them in 27 matlab files corresponding to the patch index